In [1]:
from qutip import *
import numpy as np
import scipy
import cmath
import matplotlib.pyplot as plt
from numpy.random import seed
from numpy.random import rand
from itertools import product
from scipy.misc import derivative
from scipy import optimize
seed(1)

In [2]:
#operator definitionis
N = 2
si = qeye(2)
sx = 0.5*sigmax()
sy = 0.5*sigmay()
sz = 0.5*sigmaz()

In [3]:
#make list of tensor products
sx_list = []
sy_list = []
sz_list = []

for n in range(N):
    op_list = []
    for m in range(N):
        op_list.append(si)

    op_list[n] = sx
    sx_list.append(tensor(op_list))

    op_list[n] = sy
    sy_list.append(tensor(op_list))

    op_list[n] = sz
    sz_list.append(tensor(op_list))


In [4]:
#Construct Hamiltonian
B = 0
A0 = 1
H = B*sz_list[0] 
for n in range(N-1):
    H += A0*sz_list[0]*sz_list[n+1] + A0*sx_list[0]*sx_list[n+1] + A0*sy_list[0]*sy_list[n+1]
print(H)

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.25  0.    0.    0.  ]
 [ 0.   -0.25  0.5   0.  ]
 [ 0.    0.5  -0.25  0.  ]
 [ 0.    0.    0.    0.25]]


In [5]:
#Find Groundstate
H.groundstate()

(-0.7500000000000001,
 Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
 Qobj data =
 [[ 0.        ]
  [ 0.70710678]
  [-0.70710678]
  [ 0.        ]])

In [6]:
#Make basis
Sbasis = []
for i in range(N):
    for j in range(N):
        Sbasis.append(tensor([basis(N,i),basis(N,j)]))
print(Sbasis)

[Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]], Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]], Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [1.]
 [0.]], Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [1.]]]


In [7]:
#get sz values for basis states
sz = np.zeros((2**N,N), dtype = complex)
a = [[1 for j in range(N)] for i in range(2**N)]
for i in range(2**N):
    for j in range(N):
        #matrix element <bra|Sz|ket>
        sz[i][j] = sz_list[j].matrix_element(Sbasis[i],Sbasis[i])

print(sz)

[[ 0.5+0.j  0.5+0.j]
 [ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]
 [-0.5+0.j -0.5+0.j]]


In [8]:
#Define RBM Parameters
M = 2
alp = M/N
#Number of parameters 
num = N+M+N*M

#par = np.random.rand(num)
par = np.array([0,0,0,0,0,0,0,0,0,0,0.5*np.pi,0.25*np.pi,0.5*np.pi,-1.5*np.pi,0,-0.5*np.pi])




In [9]:
#Function to give RBM wavefuntion
def RBM_ansatz(par, Sbasis, N, M,sz):
    #make parmeters complex
    parC = np.vectorize(complex)(par[:num],par[num:])
    a = parC[:N]
    b = parC[N:N+M]
    W = parC[N+M:].reshape(N,M)
    expTerm = np.zeros(2**N, dtype = complex)
    coshTerm = np.zeros((M,2**N), dtype = complex)
    hidProduct = np.zeros(2**N, dtype = complex)
    psiMValues = np.zeros(2**N, dtype = complex)
    psiM = 0*Sbasis[0]

    for i in range(2**N):
        for m in range(M):
            coshTerm[m][i] = 2*np.cosh(np.dot(W[m],sz[i]) + b[m])
    hidProduct = np.prod(coshTerm, axis = 0) 
    print(coshTerm)
    
    for i in range(2**N):
        expTerm[i] = np.exp(np.dot(a,sz[i]))
        psiMValues[i] = expTerm[i]*hidProduct[i]
        psiM += psiMValues[i]*Sbasis[i]
    psiNorm = psiM.unit()
    return psiNorm

test = RBM_ansatz(par, Sbasis, N, M,sz)
test


[[ 2.0000000e+00+0.j -3.6739404e-16-0.j  1.2246468e-16+0.j
  -2.0000000e+00+0.j]
 [ 2.0000000e+00+0.j  1.2246468e-16+0.j  2.0000000e+00+0.j
   1.2246468e-16+0.j]]


Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]]

In [10]:
#Variation Energy Definition
def varEnergy(par, Sbasis, N, M,sz, H):
    psiM = RBM_ansatz(par, Sbasis, N, M, sz)
    E = expect(H,psiM)
    norm = psiM.norm()**2
    Enorm = E/norm
    return Enorm

varEnergy(par, Sbasis, N, M,sz, H)

[[ 2.0000000e+00+0.j -3.6739404e-16-0.j  1.2246468e-16+0.j
  -2.0000000e+00+0.j]
 [ 2.0000000e+00+0.j  1.2246468e-16+0.j  2.0000000e+00+0.j
   1.2246468e-16+0.j]]


0.25

In [11]:
#Energy Partials
def gradEnergy(par, Sbasis, N, M,sz, H):
    eps = 0.00001
    return optimize.approx_fprime(par,varEnergy,eps,Sbasis, N, M,sz, H)

In [14]:
min = scipy.optimize.minimize(varEnergy,par,args=(Sbasis, N, M,sz, H), method='Nelder-Mead')


[[ 2.0000000e+00+0.j -3.6739404e-16-0.j  1.2246468e-16+0.j
  -2.0000000e+00+0.j]
 [ 2.0000000e+00+0.j  1.2246468e-16+0.j  2.0000000e+00+0.j
   1.2246468e-16+0.j]]
[[ 2.0000000e+00+0.j -3.6739404e-16-0.j  1.2246468e-16+0.j
  -2.0000000e+00+0.j]
 [ 2.0000000e+00+0.j  1.2246468e-16+0.j  2.0000000e+00+0.j
   1.2246468e-16+0.j]]
[[ 2.0000000e+00+0.j -3.6739404e-16-0.j  1.2246468e-16+0.j
  -2.0000000e+00+0.j]
 [ 2.0000000e+00+0.j  1.2246468e-16+0.j  2.0000000e+00+0.j
   1.2246468e-16+0.j]]
[[ 2.00000006e+00+0.00000000e+00j -3.67394051e-16-5.00000005e-04j
   1.22464684e-16-5.00000005e-04j -2.00000006e+00+6.12323406e-20j]
 [ 2.00000000e+00+0.00000000e+00j  1.22464680e-16+0.00000000e+00j
   2.00000000e+00+0.00000000e+00j  1.22464680e-16+0.00000000e+00j]]
[[ 2.00000000e+00+0.j     -3.67394040e-16-0.j      1.22464680e-16+0.j
  -2.00000000e+00+0.j    ]
 [ 2.00000006e+00+0.j      1.22464684e-16+0.0005j  2.00000006e+00+0.j
   1.22464684e-16+0.0005j]]
[[ 2.00000002e+00+0.00000000e+00j -3.67394043e-16

[[ 1.99167676+1.44620492e-05j  0.33131579+1.10399723e-04j
  -0.22507814+1.01698152e-04j -1.97903764+8.03027785e-06j]
 [ 1.99996811+2.62914091e-07j  0.02428966+2.10384189e-05j
   1.99996861+2.19163781e-07j  0.0243773 +1.35949130e-05j]]
[[ 1.9909232 +5.92912667e-06j  0.34243722+6.13064089e-05j
  -0.23329281-1.61676653e-04j -1.97740382-2.44129465e-05j]
 [ 1.99997343-2.11890832e-07j  0.01675155+6.35175435e-05j
   1.99997368-1.73251201e-07j  0.01679911+7.08577529e-05j]]
[[ 1.99163277-3.60766984e-06j  0.31991734-1.01102296e-05j
  -0.241831  -5.88229025e-05j -1.9828789 -3.91944289e-06j]
 [ 1.99997808+1.19832275e-07j  0.01498388+9.91054104e-05j
   1.999978  -6.67822636e-07j  0.01496704-6.88535760e-05j]]
[[ 1.98939926-9.48081678e-07j  0.34946075-3.57317783e-06j
  -0.24896225-6.35699203e-05j -1.97638219-8.96018929e-06j]
 [ 1.99996318+3.63469610e-07j  0.02149141-9.18675662e-05j
   1.99996332+5.08876857e-07j  0.02151437-6.77484375e-05j]]
[[ 1.98823539+5.03850333e-07j  0.34729865+5.73291138e-05j
  

[[ 1.69546521+2.60647873e-04j  1.03471144+5.55759444e-04j
  -1.35698812+8.85392782e-05j -1.87379551+9.73843662e-05j]
 [ 1.99997679+1.45439143e-06j  0.13958381-5.53871688e-04j
   1.9999772 -1.36708379e-06j  0.13949829+3.26235785e-05j]]
[[ 1.65379203+2.29780774e-04j  1.01846387+6.73764946e-04j
  -1.40349695-1.84434270e-05j -1.87621305+1.20660983e-04j]
 [ 1.99976145+5.65735638e-06j  0.11980818-5.66549385e-04j
   1.99976369-3.78894218e-06j  0.11966179+4.50976989e-05j]]
[[ 1.65924654+3.89328576e-04j  1.06966212+6.93349812e-04j
  -1.4143241 +2.05124945e-04j -1.86378896+1.49940874e-04j]
 [ 1.99998252+8.99364972e-07j  0.14020498-4.55482446e-04j
   1.99998298-8.09071022e-07j  0.14009491-4.53675740e-05j]]
[[ 1.6308655 +4.43798209e-04j  1.10062987+7.96717703e-04j
  -1.43307234+2.32383980e-04j -1.84115684+2.03347067e-04j]
 [ 1.99982704+3.92886721e-06j  0.16373771-6.71089610e-04j
   1.99982686-7.50792094e-06j  0.16375558+1.95189487e-04j]]
[[ 1.47374701+7.21581634e-04j  1.26526455+9.96473483e-04j
  

[[ 0.12913146+2.86330560e-03j  1.96895057+6.45017788e-04j
  -1.77134771-6.93577723e-04j -1.32464054+1.72255449e-03j]
 [ 1.99959971+2.40006781e-05j  0.45833537-2.68180086e-03j
   1.99960523-2.23339753e-05j  0.4580636 -4.20825648e-04j]]
[[-0.40221634+3.44976194e-03j  1.99066619-4.47942107e-04j
  -1.27212683-1.48702243e-03j -1.09980218+2.54410935e-03j]
 [ 1.99896294+6.18087799e-05j  0.56424197-3.77596557e-03j
   1.99897085-4.16140588e-05j  0.56398993-6.90807498e-04j]]
[[ 0.14689676+2.70762600e-03j  1.97227168+6.02345680e-04j
  -1.76953312-6.88275992e-04j -1.32633093+1.79005218e-03j]
 [ 1.99885264+4.45575892e-05j  0.45518146-2.97124773e-03j
   1.99886095-3.98485928e-05j  0.45493937-5.40689120e-04j]]
[[-0.39494623+3.49763882e-03j  1.99277826-3.93463938e-04j
  -1.29037641-1.46853836e-03j -1.10558351+2.48889203e-03j]
 [ 1.99931951+4.82933848e-05j  0.56261692-3.62699787e-03j
   1.99932621-3.29343891e-05j  0.56235217-6.34157488e-04j]]
[[-0.5988175 +3.62426705e-03j  1.96550525-9.18330346e-04j
  

   1.99988776-1.55582732e-05j  0.69443738-8.13845433e-04j]]
[[-0.01109891+5.20633672e-03j  1.88039671-1.50625474e-03j
  -1.44683642-2.96888307e-03j -0.79523443+3.22580377e-03j]
 [ 1.99994051-1.43190312e-05j  0.75781494-3.76761750e-03j
   1.99994572+1.11284692e-05j  0.75848318-6.70943127e-04j]]
[[ 0.00841226+5.99565066e-03j  1.73087035-2.40789770e-03j
  -1.23622631-4.07480943e-03j -0.74904687+3.70319257e-03j]
 [ 2.00000114-3.35427805e-07j  0.77510747-4.29328109e-03j
   2.00000042-1.37473918e-07j  0.77558459-1.09020295e-03j]]
[[-0.08890671+6.19250893e-03j  1.68631522-2.66877974e-03j
  -1.17323028-4.45158451e-03j -0.65146658+4.02922242e-03j]
 [ 1.99927381-5.50914064e-05j  0.82332242-4.17091944e-03j
   1.99929229+3.62951740e-05j  0.82396875-1.06574045e-03j]]
[[-0.15104645+6.35008100e-03j  1.62334319-2.96145201e-03j
  -1.063118  -4.71340030e-03j -0.61196836+4.06173196e-03j]
 [ 2.00000093+9.51798386e-07j  0.83293958-4.38905810e-03j
   2.00000014-1.33442572e-06j  0.83353195-8.93105099e-04j]]


...

   1.73493310e+00-0.00163581j  1.74734658e+00-0.00010643j]]
[[-6.15955414e-04+2.25951562e-03j  1.93654707e+00-1.66020045e-03j
  -1.93684337e+00+1.32754712e-03j -5.86358400e-04-3.57818320e-03j]
 [ 1.74281158e+00-8.95256885e-04j  1.75502808e+00+6.01824410e-04j
   1.74639603e+00-1.53664497e-03j  1.75852997e+00-3.47808024e-05j]]
[[ 1.42118464e-06+0.00249644j  1.93080095e+00-0.00161595j
  -1.93088620e+00+0.00141677j -3.36622844e-04-0.00325685j]
 [ 1.72464623e+00-0.00091639j  1.73723057e+00+0.00060158j
   1.72837717e+00-0.00164972j  1.74087919e+00-0.00012541j]]
[[ 3.65865472e-04+2.48858888e-03j  1.92047021e+00-1.75332313e-03j
  -1.92059778e+00+1.52697186e-03j -8.31109310e-04-3.29489490e-03j]
 [ 1.72881376e+00-9.22871621e-04j  1.74123481e+00+5.68178662e-04j
   1.73254678e+00-1.57346017e-03j  1.74488539e+00-7.78038777e-05j]]
[[-6.40302955e-04+1.86298737e-03j  1.90573027e+00-2.02334094e-03j
  -1.90565338e+00+1.44204355e-03j  8.76457023e-04-3.78061846e-03j]
 [ 1.71793913e+00-8.47058253e-04j  1.7

[[-2.82048498e-04+0.00245154j  1.95541308e+00-0.00130076j
  -1.95541858e+00+0.00112548j  2.44601938e-04-0.00328571j]
 [ 1.72500469e+00-0.00089754j  1.73812513e+00+0.00065848j
   1.72865623e+00-0.00175837j  1.74169216e+00-0.00019248j]]
[[-4.53447435e-04+0.00249589j  1.96390129e+00-0.00116502j
  -1.96391031e+00+0.00102204j  3.94449105e-04-0.00325094j]
 [ 1.72628050e+00-0.00089825j  1.73971742e+00+0.00065445j
   1.72990162e+00-0.00178111j  1.74325223e+00-0.00021739j]]
[[-3.25441061e-04+0.00242173j  1.94428861e+00-0.00149153j
  -1.94441888e+00+0.001267j   -2.42268889e-04-0.00337326j]
 [ 1.73363925e+00-0.00090552j  1.74630212e+00+0.00060551j
   1.73727388e+00-0.00162982j  1.74985304e+00-0.00011202j]]
[[-4.19391586e-04+0.00241147j  1.97250162e+00-0.00103477j
  -1.97259650e+00+0.00087958j -1.71272844e-04-0.00334102j]
 [ 1.73048171e+00-0.00088367j  1.74351623e+00+0.00068299j
   1.73403492e+00-0.00180259j  1.74698569e+00-0.00022445j]]
[[-6.71513125e-04+0.00240961j  1.98906389e+00-0.00065607j
  

[[-2.45196674e-04+0.00265022j  1.99673042e+00-0.0003498j
  -1.99676957e+00+0.00031814j -4.67465526e-04-0.00316849j]
 [ 1.74044479e+00-0.00088953j  1.75427001e+00+0.00064164j
   1.74382567e+00-0.00187921j  1.75756263e+00-0.00033221j]]
[[-6.44402947e-05+0.00250819j  1.97945316e+00-0.00090496j
  -1.97945159e+00+0.00078793j  5.86422092e-05-0.00332647j]
 [ 1.74139201e+00-0.0008964j   1.75519763e+00+0.00061352j
   1.74486913e+00-0.00172358j  1.75858428e+00-0.00020206j]]
[[-2.94614222e-04+0.00243383j  1.99845000e+00-0.0002526j
  -1.99846289e+00+0.00021307j -1.05562735e-04-0.00340725j]
 [ 1.74854350e+00-0.00086429j  1.76250527e+00+0.00068574j
   1.75183241e+00-0.00186535j  1.76570428e+00-0.00029822j]]
[[-1.45294244e-04+0.00251203j  1.98671312e+00-0.00073912j
  -1.98676365e+00+0.00063913j -3.16316879e-04-0.00336948j]
 [ 1.74967883e+00-0.00089361j  1.76341170e+00+0.00059395j
   1.75307249e+00-0.00169549j  1.76671439e+00-0.00019631j]]
[[ 7.65629714e-05+0.00238305j  1.99118753e+00-0.00060998j
  -1

[[-1.36942316e-04+0.00261073j  1.98964996e+00-0.00063933j
  -1.98966477e+00+0.00057251j -2.78526383e-05-0.00326351j]
 [ 1.74702702e+00-0.00090055j  1.76108549e+00+0.00058524j
   1.75042950e+00-0.00173214j  1.76439561e+00-0.00023357j]]
[[-8.53024054e-07+0.00262551j  1.98903811e+00-0.00065723j
  -1.98905225e+00+0.0005912j  -1.52100623e-04-0.00325264j]
 [ 1.74732365e+00-0.00090307j  1.76128570e+00+0.00057667j
   1.75072998e+00-0.00172122j  1.76460013e+00-0.00022917j]]
[[ 4.98304730e-05+0.00266023j  1.98805485e+00-0.0006853j
  -1.98806749e+00+0.0006225j  -1.81252820e-04-0.00323202j]
 [ 1.74854270e+00-0.00090888j  1.76249100e+00+0.0005556j
   1.75195262e+00-0.00169466j  1.76580863e+00-0.00021859j]]
[[-2.65532531e-05+0.0026286j   1.99349009e+00-0.00050582j
  -1.99351130e+00+0.00045505j -2.59442861e-04-0.00324787j]
 [ 1.74810850e+00-0.00089755j  1.76214018e+00+0.00059109j
   1.75147853e+00-0.00175941j  1.76541837e+00-0.0002571j ]]
[[ 5.60753266e-05+0.00258114j  1.99112745e+00-0.00059157j
  -1

[[-6.41522778e-05+0.00263641j  1.99064521e+00-0.0006021j
  -1.99065434e+00+0.00054499j -4.83183723e-05-0.00322406j]
 [ 1.74482971e+00-0.00090102j  1.75882300e+00+0.00059179j
   1.74824022e+00-0.00175975j  1.76214216e+00-0.00025375j]]
[[-7.02086103e-06+0.00267228j  1.98957792e+00-0.00063604j
  -1.98959061e+00+0.00058097j -1.32976414e-04-0.00320828j]
 [ 1.74690799e+00-0.00090762j  1.76086753e+00+0.00056548j
   1.75031864e+00-0.00172292j  1.76418636e+00-0.00023758j]]
[[-8.31959527e-05+0.00263736j  1.99116186e+00-0.00058557j
  -1.99117811e+00+0.0005298j  -1.08361644e-04-0.00322518j]
 [ 1.74531892e+00-0.00090059j  1.75923723e+00+0.00059137j
   1.74872304e+00-0.00176001j  1.76255045e+00-0.00025492j]]
[[-2.53606418e-05+0.00262625j  1.99243749e+00-0.00054226j
  -1.99244212e+00+0.00048922j -4.86465811e-05-0.00323397j]
 [ 1.74543466e+00-0.0008976j   1.75945858e+00+0.00060001j
   1.74882753e+00-0.00177431j  1.76276007e+00-0.00026294j]]


In [20]:
min.message

'Maximum number of function evaluations has been exceeded.'

In [16]:
#Ground State
final=RBM_ansatz(min.x, Sbasis, N, M,sz)
final.unit()

[[ 1.31770384e-05+0.0025982j   1.98752058e+00-0.0007042j
  -1.98752734e+00+0.00062813j -9.18090664e-05-0.00327791j]
 [ 1.74733169e+00-0.00090162j  1.76115505e+00+0.00058005j
   1.75074402e+00-0.00171185j  1.76447625e+00-0.00021832j]]


Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[ 4.26225454e-06+0.0009212j ]
 [ 7.07133320e-01-0.00030538j]
 [-7.07078328e-01+0.00062712j]
 [-3.40422833e-05-0.00117009j]]

In [18]:
#Minimum Energy
varEnergy(min2.x, Sbasis, N, M,sz, H)

[[ 1.31770384e-05+0.0025982j   1.98752058e+00-0.0007042j
  -1.98752734e+00+0.00062813j -9.18090664e-05-0.00327791j]
 [ 1.74733169e+00-0.00090162j  1.76115505e+00+0.00058005j
   1.75074402e+00-0.00171185j  1.76447625e+00-0.00021832j]]


-0.7499977278382108